In [4]:
import pandas as pd

matches = pd.read_csv("Data/Matches.csv", parse_dates=["MatchDate"])
elo = pd.read_csv("Data/EloRatings.csv", parse_dates=["date"])

matches.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_14860\2730841688.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv("Data/Matches.csv", parse_dates=["MatchDate"])


,Division,MatchDate,MatchTime,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,...,MaxUnder25,HandiSize,HandiHome,HandiAway,C_LTH,C_LTA,C_VHD,C_VAD,C_HTB,C_PHB
0,F1,2000-07-28,NaN,Marseille,Troyes,1686.34,1586.57,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F1,2000-07-28,NaN,Paris SG,Strasbourg,1714.89,1642.51,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F2,2000-07-28,NaN,Wasquehal,Nancy,1465.08,1633.80,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F1,2000-07-29,NaN,Auxerre,Sedan,1635.58,1624.22,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F1,2000-07-29,NaN,Bordeaux,Metz,1734.34,1673.11,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
elo.head()

,date,club,country,elo
0,2000-07-01,Aachen,GER,1453.60
1,2000-07-01,Aalborg,DEN,1482.61
2,2000-07-01,Aalst,BEL,1337.53
3,2000-07-01,Aarhus,DEN,1381.46
4,2000-07-01,Aberdeen,SCO,1360.43


In [6]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230557 entries, 0 to 230556
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Division     230557 non-null  object        
 1   MatchDate    230557 non-null  datetime64[ns]
 2   MatchTime    99072 non-null   object        
 3   HomeTeam     230557 non-null  object        
 4   AwayTeam     230557 non-null  object        
 5   HomeElo      141597 non-null  float64       
 6   AwayElo      141528 non-null  float64       
 7   Form3Home    229057 non-null  float64       
 8   Form5Home    229057 non-null  float64       
 9   Form3Away    229057 non-null  float64       
 10  Form5Away    229057 non-null  float64       
 11  FTHome       230554 non-null  float64       
 12  FTAway       230554 non-null  float64       
 13  FTResult     230554 non-null  object        
 14  HTHome       175977 non-null  float64       
 15  HTAway       175977 non-null  floa

In [9]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230557 entries, 0 to 230556
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Division     230557 non-null  object        
 1   MatchDate    230557 non-null  datetime64[ns]
 2   MatchTime    99072 non-null   object        
 3   HomeTeam     230557 non-null  object        
 4   AwayTeam     230557 non-null  object        
 5   HomeElo      141597 non-null  float64       
 6   AwayElo      141528 non-null  float64       
 7   Form3Home    229057 non-null  float64       
 8   Form5Home    229057 non-null  float64       
 9   Form3Away    229057 non-null  float64       
 10  Form5Away    229057 non-null  float64       
 11  FTHome       230554 non-null  float64       
 12  FTAway       230554 non-null  float64       
 13  FTResult     230554 non-null  object        
 14  HTHome       175977 non-null  float64       
 15  HTAway       175977 non-null  floa

In [10]:
matches.isna().sum().sort_values(ascending=False).head(15)


MatchTime      131485
C_LTA          117955
C_VHD          117955
C_VAD          117955
C_LTH          117955
C_HTB          117955
C_PHB          117955
HomeTarget     116628
AwayTarget     116625
HomeFouls      116584
AwayFouls      116584
HomeCorners    116194
AwayCorners    116194
HomeShots      115822
AwayShots      115819
dtype: int64

In [11]:
df = matches.copy()

# Keep only rows with result
df = df.dropna(subset=["FTResult"])

# Fill numeric NaNs with median
num_cols = df.select_dtypes(include="number").columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())


In [12]:
df["EloDiff"] = df["HomeElo"] - df["AwayElo"]
df["Form3Diff"] = df["Form3Home"] - df["Form3Away"]
df["Form5Diff"] = df["Form5Home"] - df["Form5Away"]

df["ShotDiff"] = df["HomeShots"] - df["AwayShots"]
df["TargetDiff"] = df["HomeTarget"] - df["AwayTarget"]

df["FoulDiff"] = df["HomeFouls"] - df["AwayFouls"]
df["CornerDiff"] = df["HomeCorners"] - df["AwayCorners"]

df["CardDiff"] = (df["HomeYellow"] + df["HomeRed"]) - (df["AwayYellow"] + df["AwayRed"])

df["GoalDiff"] = df["FTHome"] - df["FTAway"]


In [13]:
df["ResultLabel"] = df["FTResult"].map({"H":0, "D":1, "A":2})
df["ResultLabel"].value_counts(normalize=True)


ResultLabel
0    0.446199
2    0.288696
1    0.265105
Name: proportion, dtype: float64

In [14]:
features = [
    "EloDiff","Form3Diff","Form5Diff",
    "ShotDiff","TargetDiff",
    "FoulDiff","CornerDiff","CardDiff",
    "OddHome","OddDraw","OddAway"
]

X = df[features]
y = df["ResultLabel"]


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
pred = model.predict(X_test)

print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.55      0.85      0.67     20575
           1       0.36      0.03      0.05     12224
           2       0.52      0.53      0.52     13312

    accuracy                           0.54     46111
   macro avg       0.48      0.47      0.41     46111
weighted avg       0.49      0.54      0.46     46111



In [16]:
#Feature importance
import pandas as pd

imp = pd.Series(model.feature_importances_, index=features)\
        .sort_values(ascending=False)

imp


TargetDiff    0.248768
OddAway       0.189164
OddHome       0.176401
CornerDiff    0.079631
OddDraw       0.070460
EloDiff       0.065668
ShotDiff      0.051042
Form5Diff     0.035645
FoulDiff      0.033647
Form3Diff     0.025296
CardDiff      0.024278
dtype: float64

In [19]:
#Predictions
df["PredictedResult"] = model.predict(X)
df["PredictedLabel"] = df["PredictedResult"].map({0:"Home",1:"Draw",2:"Away"})


In [18]:
#Analytics Columns
df["TotalGoals"] = df["FTHome"] + df["FTAway"]
df["Over25Label"] = (df["TotalGoals"] > 2.5).astype(int)

df["ShotAccuracyHome"] = df["HomeTarget"] / df["HomeShots"]
df["ShotAccuracyAway"] = df["AwayTarget"] / df["AwayShots"]


In [21]:
#Export for tableau
df.to_csv("Data/Football_Tableau.csv", index=False)
